In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import dpkt

import collections as clt

In [2]:
def resize(arr, size):
    if len(arr) >= size:
        return arr[0:size]
    return np.pad(arr,(0,size - len(arr)))

In [3]:
data_frame = list()

local_mac = b'\xc8\x3d\xd4\x99\x71\x67'
resize_len = 1000
input_path = 'F:/data_edge/2_pcap_ow/'

count_url = 1000
count_repeat = 1

for A in range(100, count_url + 100):
    
    for B in range(count_repeat):
        
        file_name = input_path + str(A) + '-' + str(B) + '.pcap'
        
        print('\rprocessing:' + str(A) + '-' + str(B), end='')
        
        try:
            with open(file_name, 'rb') as f:

                pcap = dpkt.pcap.Reader(f)

                data = list()
                data.append(A)
                data.append(B)

                for ptime, pdata in pcap:

                    try:
                        eth = dpkt.ethernet.Ethernet(pdata)
                        mac_src = eth.src
                        mac_dst = eth.dst

                        if mac_src == local_mac:
                            data.append(len(pdata))
                        elif mac_dst == local_mac:
                            data.append(-len(pdata))
                        else:
                            pass
                    except:
                        print('\ttail')

                data_list = np.array(data)
                data_list = resize(data_list, resize_len + 2)
                data_list = np.array([data_list])

                if len(data_frame) == 0:
                    data_frame = data_list
                else:
                    data_frame = np.append(data_frame, data_list, axis=0)
        except Exception as e:
            print('\topenerr')
            pass
                
print('\ndone!')

processing:1086-0	tail
processing:1099-0
done!


In [4]:
data_frame.shape

(1000, 1002)

In [5]:
out_path = 'F:/data_edge/3_data/'

df_data = pd.DataFrame(data_frame)
df_data.to_csv(out_path + 'data_ow.csv', index=None, header=None)